# اوپن اے آئی ماڈلز کی فائن ٹیوننگ

یہ نوٹ بک اوپن اے آئی کی [فائن ٹیوننگ](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) دستاویزات میں دی گئی موجودہ رہنمائی پر مبنی ہے۔

فائن ٹیوننگ آپ کی ایپلیکیشن کے لیے بنیادی ماڈلز کی کارکردگی کو بہتر بناتی ہے، کیونکہ اس میں ماڈل کو اضافی ڈیٹا اور اس مخصوص استعمال یا منظرنامے سے متعلق سیاق و سباق کے ساتھ دوبارہ تربیت دی جاتی ہے۔ یاد رہے کہ پرامپٹ انجینئرنگ کی تکنیکیں جیسے _فیوشاٹ لرننگ_ اور _ریٹریول آگمینٹڈ جنریشن_ آپ کو متعلقہ ڈیٹا کے ساتھ ڈیفالٹ پرامپٹ کو بہتر بنانے کی سہولت دیتی ہیں تاکہ معیار میں بہتری آئے۔ تاہم، یہ طریقے ہدف بنائے گئے بنیادی ماڈل کی زیادہ سے زیادہ ٹوکن ونڈو سائز کی حد تک محدود ہیں۔

فائن ٹیوننگ کے ذریعے، ہم دراصل ماڈل کو مطلوبہ ڈیٹا کے ساتھ دوبارہ تربیت دے رہے ہیں (جس سے ہم زیادہ مثالیں استعمال کر سکتے ہیں جتنی کہ زیادہ سے زیادہ ٹوکن ونڈو میں آ سکتی ہیں) - اور ماڈل کا ایک _کسٹم_ ورژن تعینات کر رہے ہیں جسے اب انفیرینس کے وقت مثالیں فراہم کرنے کی ضرورت نہیں رہتی۔ اس سے نہ صرف ہمارے پرامپٹ ڈیزائن کی افادیت میں اضافہ ہوتا ہے (ہم ٹوکن ونڈو کو دیگر مقاصد کے لیے زیادہ لچک کے ساتھ استعمال کر سکتے ہیں) بلکہ ممکنہ طور پر ہمارے اخراجات میں بھی کمی آتی ہے (کیونکہ انفیرینس کے وقت ماڈل کو بھیجے جانے والے ٹوکنز کی تعداد کم ہو جاتی ہے)۔

فائن ٹیوننگ کے 4 مراحل ہیں:
1. تربیتی ڈیٹا تیار کریں اور اپ لوڈ کریں۔
1. تربیتی جاب چلائیں تاکہ فائن ٹیونڈ ماڈل حاصل ہو سکے۔
1. فائن ٹیونڈ ماڈل کا جائزہ لیں اور معیار کے لیے اسے بہتر بنائیں۔
1. جب مطمئن ہوں تو فائن ٹیونڈ ماڈل کو انفیرینس کے لیے تعینات کریں۔

یاد رہے کہ تمام بنیادی ماڈلز فائن ٹیوننگ کو سپورٹ نہیں کرتے - تازہ ترین معلومات کے لیے [اوپن اے آئی کی دستاویزات دیکھیں](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)۔ آپ پہلے سے فائن ٹیونڈ ماڈل کو بھی دوبارہ فائن ٹیون کر سکتے ہیں۔ اس ٹیوٹوریل میں، ہم `gpt-35-turbo` کو فائن ٹیوننگ کے لیے اپنا ہدف بنیادی ماڈل منتخب کریں گے۔

---


### مرحلہ 1.1: اپنا ڈیٹاسیٹ تیار کریں

آئیے ایک چیٹ بوٹ بناتے ہیں جو آپ کو عناصر کے پیریاوڈک ٹیبل کو سمجھنے میں مدد دے، اور ہر عنصر کے بارے میں سوالات کے جوابات ایک لیمیرک نظم کی صورت میں دے۔ اس سادہ ٹیوٹوریل میں، ہم صرف ایک ڈیٹاسیٹ بنائیں گے جس میں کچھ نمونہ جوابات ہوں گے جو ڈیٹا کی متوقع شکل کو ظاہر کرتے ہیں۔ حقیقی دنیا کے استعمال میں، آپ کو بہت زیادہ مثالوں کے ساتھ ایک ڈیٹاسیٹ تیار کرنا ہوگا۔ اگر آپ کے شعبے کے لیے کوئی اوپن ڈیٹاسیٹ دستیاب ہے تو آپ اسے بھی استعمال کر سکتے ہیں اور اسے فائن ٹیوننگ کے لیے دوبارہ فارمیٹ کر سکتے ہیں۔

چونکہ ہم `gpt-35-turbo` پر توجہ مرکوز کر رہے ہیں اور ایک ہی بار میں جواب (chat completion) چاہتے ہیں، اس لیے ہم [اس تجویز کردہ فارمیٹ](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) کے مطابق مثالیں بنا سکتے ہیں جو OpenAI چیٹ کمپلیشن کی ضروریات کو ظاہر کرتا ہے۔ اگر آپ کو کئی بار کے مکالمے کی ضرورت ہے تو آپ [ملٹی ٹرن مثال کے فارمیٹ](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) کا استعمال کریں گے، جس میں `weight` پیرامیٹر شامل ہوتا ہے تاکہ یہ بتایا جا سکے کہ کون سے پیغامات فائن ٹیوننگ میں شامل کیے جائیں یا نہیں۔

ہم یہاں اپنے ٹیوٹوریل کے لیے سادہ سنگل ٹرن فارمیٹ استعمال کریں گے۔ ڈیٹا [jsonl فارمیٹ](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) میں ہے، جس میں ہر لائن پر ایک ریکارڈ ہوتا ہے، اور ہر ریکارڈ ایک JSON آبجیکٹ کی صورت میں ہوتا ہے۔ نیچے دی گئی مثال میں 2 ریکارڈز دکھائے گئے ہیں - مکمل نمونہ سیٹ (10 مثالیں) کے لیے [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) دیکھیں، جو ہم اپنے فائن ٹریننگ ٹیوٹوریل میں استعمال کریں گے۔ **نوٹ:** ہر ریکارڈ _ضرور_ ایک ہی لائن میں ہونا چاہیے (جیسا کہ فارمیٹڈ JSON فائل میں عام طور پر لائنوں میں تقسیم کیا جاتا ہے، ویسا نہیں)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

حقیقی دنیا کے استعمال میں اچھے نتائج کے لیے آپ کو بہت زیادہ مثالوں کی ضرورت ہوگی - اس میں توازن یہ ہے کہ جتنی زیادہ مثالیں ہوں گی، جواب کی کوالٹی بہتر ہوگی لیکن فائن ٹیوننگ میں وقت اور لاگت بھی زیادہ ہوگی۔ ہم یہاں چھوٹا سیٹ استعمال کر رہے ہیں تاکہ عمل کو جلد مکمل کر کے آپ کو سمجھا سکیں۔ مزید پیچیدہ فائن ٹیوننگ ٹیوٹوریل کے لیے [یہ OpenAI Cookbook مثال](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) دیکھیں۔


### مرحلہ 1.2 اپنا ڈیٹاسیٹ اپ لوڈ کریں

ڈیٹا کو فائلز API کے ذریعے اپ لوڈ کریں [جیسا کہ یہاں بیان کیا گیا ہے](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)۔ نوٹ کریں کہ اس کوڈ کو چلانے کے لیے آپ کو پہلے یہ اقدامات مکمل کرنے ہوں گے:
 - `openai` پائتھن پیکیج انسٹال کیا ہو (یقینی بنائیں کہ آپ ورژن >=0.28.0 استعمال کر رہے ہیں تاکہ تازہ ترین فیچرز دستیاب ہوں)
 - `OPENAI_API_KEY` انوائرمنٹ ویری ایبل کو اپنے OpenAI API کلید کے ساتھ سیٹ کیا ہو
مزید جاننے کے لیے، اس کورس کے لیے فراہم کردہ [سیٹ اپ گائیڈ](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) دیکھیں۔

اب، کوڈ چلائیں تاکہ آپ اپنی مقامی JSONL فائل سے اپ لوڈ کے لیے ایک فائل بنا سکیں۔


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### مرحلہ 2.1: SDK کے ساتھ فائن ٹیوننگ جاب بنائیں


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### مرحلہ 2.2: جاب کی حالت چیک کریں

یہاں کچھ کام ہیں جو آپ `client.fine_tuning.jobs` API کے ساتھ کر سکتے ہیں:
- `client.fine_tuning.jobs.list(limit=<n>)` - آخری n فائن ٹیوننگ جابز کی فہرست دیکھیں
- `client.fine_tuning.jobs.retrieve(<job_id>)` - کسی خاص فائن ٹیوننگ جاب کی تفصیلات حاصل کریں
- `client.fine_tuning.jobs.cancel(<job_id>)` - فائن ٹیوننگ جاب کو منسوخ کریں
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - جاب کے n تک ایونٹس کی فہرست دیکھیں
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

اس عمل کا پہلا مرحلہ _ٹریننگ فائل کی تصدیق کرنا_ ہے تاکہ یہ یقینی بنایا جا سکے کہ ڈیٹا درست فارمیٹ میں ہے۔


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### مرحلہ 2.3: پیش رفت کو مانیٹر کرنے کے لیے ایونٹس کو ٹریک کریں


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### مرحلہ 2.4: اوپن اے آئی ڈیش بورڈ میں اسٹیٹس دیکھیں


آپ OpenAI کی ویب سائٹ پر جا کر اور پلیٹ فارم کے _فائن ٹیوننگ_ سیکشن میں جا کر بھی اسٹیٹس دیکھ سکتے ہیں۔ یہاں آپ کو موجودہ جاب کی صورتحال نظر آئے گی، اور آپ پچھلے جابز کی ہسٹری بھی ٹریک کر سکتے ہیں۔ اس اسکرین شاٹ میں آپ دیکھ سکتے ہیں کہ پہلی بار جاب ناکام ہوئی تھی، اور دوسری بار کامیاب رہی۔ وضاحت کے لیے، یہ اس وقت ہوا جب پہلی بار JSON فائل میں ریکارڈز غلط فارمیٹ میں تھے - جب یہ مسئلہ حل کیا گیا تو دوسری بار جاب کامیابی سے مکمل ہوئی اور ماڈل استعمال کے لیے دستیاب ہو گیا۔

![فائن ٹیوننگ جاب اسٹیٹس](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ur.png)


آپ مزید اسٹیٹس پیغامات اور میٹرکس کو نیچے اسکرول کر کے بصری ڈیش بورڈ میں دیکھ سکتے ہیں جیسا کہ نیچے دکھایا گیا ہے:

| پیغامات | میٹرکس |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ur.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ur.png)|


### مرحلہ 3.1: آئی ڈی حاصل کریں اور کوڈ میں فائن ٹیونڈ ماڈل کو ٹیسٹ کریں


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### مرحلہ 3.2: فائن ٹیونڈ ماڈل کو پلے گراؤنڈ میں لوڈ اور ٹیسٹ کریں

اب آپ فائن ٹیونڈ ماڈل کو دو طریقوں سے آزما سکتے ہیں۔ سب سے پہلے، آپ پلے گراؤنڈ پر جا کر ماڈلز کے ڈراپ ڈاؤن سے اپنے نئے فائن ٹیونڈ ماڈل کو منتخب کر سکتے ہیں۔ دوسرا طریقہ یہ ہے کہ فائن ٹیوننگ پینل میں دکھائے گئے "پلے گراؤنڈ" آپشن کو استعمال کریں (اوپر اسکرین شاٹ دیکھیں) جو ایک _موازنہ_ ویو کھولتا ہے، جس میں بنیادی اور فائن ٹیونڈ ماڈل دونوں کو ساتھ ساتھ دکھایا جاتا ہے تاکہ آپ جلدی سے ان کا جائزہ لے سکیں۔

![فائن ٹیوننگ جاب اسٹیٹس](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ur.png)

بس وہی سسٹم کانٹیکسٹ درج کریں جو آپ نے اپنی ٹریننگ ڈیٹا میں استعمال کیا تھا اور اپنا ٹیسٹ سوال دیں۔ آپ دیکھیں گے کہ دونوں طرف ایک ہی کانٹیکسٹ اور سوال اپڈیٹ ہو جاتے ہیں۔ موازنہ چلائیں اور آپ کو دونوں کے آؤٹ پٹ میں فرق نظر آئے گا۔ _نوٹ کریں کہ فائن ٹیونڈ ماڈل آپ کے دیے گئے فارمیٹ میں جواب دیتا ہے، جبکہ بنیادی ماڈل صرف سسٹم پرامپٹ کو فالو کرتا ہے۔_

![فائن ٹیوننگ جاب اسٹیٹس](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ur.png)

آپ دیکھیں گے کہ موازنہ میں ہر ماڈل کے لیے ٹوکن کی تعداد اور انفیرینس میں لگنے والا وقت بھی دکھایا جاتا ہے۔ **یہ خاص مثال صرف عمل کو سمجھانے کے لیے ہے، یہ کسی حقیقی ڈیٹا سیٹ یا منظرنامے کی عکاسی نہیں کرتی۔** آپ دیکھ سکتے ہیں کہ دونوں نمونوں میں ٹوکن کی تعداد ایک جیسی ہے (کیونکہ سسٹم کانٹیکسٹ اور یوزر پرامپٹ ایک جیسے ہیں)، لیکن فائن ٹیونڈ ماڈل (کسٹم ماڈل) کو انفیرینس میں زیادہ وقت لگتا ہے۔

حقیقی دنیا میں آپ اس طرح کی سادہ مثال استعمال نہیں کریں گے، بلکہ اصل ڈیٹا (مثلاً کسٹمر سروس کے لیے پروڈکٹ کیٹلاگ) پر فائن ٹیوننگ کریں گے، جہاں جواب کے معیار میں واضح فرق نظر آئے گا۔ اس _حقیقی_ تناظر میں، اگر آپ بنیادی ماڈل سے بھی اسی معیار کا جواب لینا چاہیں تو اس کے لیے زیادہ کسٹم پرامپٹ انجینئرنگ کرنا پڑے گی، جس سے ٹوکن کا استعمال اور انفیرینس کا وقت بڑھ جائے گا۔ _اسے آزمانے کے لیے، OpenAI Cookbook میں موجود فائن ٹیوننگ کی مثالیں دیکھیں اور آغاز کریں۔_



---

**اعلانِ دستبرداری**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کی پوری کوشش کرتے ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہے۔ اصل دستاویز کو اس کی اصل زبان میں مستند ماخذ سمجھا جانا چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ تجویز کیا جاتا ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر نہیں ہوگی۔
